## Импорты

In [10]:
import pandas as pd
import numpy as np
from sklearn.utils import check_array

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler 

from sklearn.model_selection import train_test_split

PATH_TO_DATA="/Users/roman/finunichamp2018/1/data/"
PATH_TO_SUBMIT="/Users/roman/finunichamp2018/1/answers_archive/"

In [2]:
X_train_lin=pd.read_csv(PATH_TO_DATA+"X_train_lin.csv")
X_test_lin=pd.read_csv(PATH_TO_DATA+"X_test_lin.csv")

X_train_tree=pd.read_csv(PATH_TO_DATA+"X_train_tree.csv")
X_test_tree=pd.read_csv(PATH_TO_DATA+"X_test_tree.csv")

X_train_xgb=pd.read_csv(PATH_TO_DATA+"X_train_xgb.csv")
X_test_xgb=pd.read_csv(PATH_TO_DATA+"X_test_xgb.csv")

X_train_cat=pd.read_csv(PATH_TO_DATA+"X_train_cat.csv")
X_test_cat=pd.read_csv(PATH_TO_DATA+"X_test_cat.csv")

y=pd.read_csv(PATH_TO_DATA+"Y.csv")
y=y["target"]

In [3]:
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [4]:
def submit(pred,name):
    sub=pd.read_csv(PATH_TO_DATA+"insvalue_sample.csv",index_col="id")
    sub["target"]=pred
    sub.to_csv(PATH_TO_SUBMIT+name+".csv")

## Обучение моделей

### KNN

In [13]:
X_train, X_test, y_train, y_test=train_test_split(X_train_tree,y,test_size=0.3, random_state=17)

knn=KNeighborsRegressor()
knn.fit(X_train,y_train)


pred=knn.predict(X_test)
MAPE(y_test, pred)

27.887601428852065

### Lin reg

In [6]:
X_train, X_test, y_train, y_test=train_test_split(X_train_lin,y,test_size=0.3, random_state=17)

linReg=LinearRegression(n_jobs=-1)
linReg.fit(X_train,y_train)


pred=linReg.predict(X_test)
MAPE(y_test, pred)

2894780.4548161249

In [6]:
linReg.fit(X_train_lin,y)
pred=linReg.predict(X_test_lin)
submit(pred,"lin_reg_dirty_data")

### SVM

In [7]:
X_train, X_test, y_train, y_test=train_test_split(X_train_lin,y,test_size=0.3, random_state=17)

svm=LinearSVR(random_state=17)
svm.fit(X_train,y_train)

pred=svm.predict(X_test)
MAPE(y_test, pred)

27.626965385280549

In [89]:
svm.fit(X_train_lin,y)
pred=svm.predict(X_test_lin)
submit(pred,"svm_dirty_data")

### Random Forest

In [8]:
X_train, X_test, y_train, y_test=train_test_split(X_train_tree,y,test_size=0.3, random_state=17)

rf=RandomForestRegressor(random_state=17)
rf.fit(X_train,y_train)

pred=rf.predict(X_test)
MAPE(y_test, pred)

9.308827486399915

In [17]:
rf.fit(X_train_tree,np.log(y))
pred=rf.predict(X_test_tree)
submit(np.exp(pred),"rf_test_clead_log")

### Xgb

In [9]:
X_train, X_test, y_train, y_test=train_test_split(X_train_xgb,y,test_size=0.3, random_state=17)

xgb=XGBRegressor(seed=17)
xgb.fit(X_train,y_train)

pred=xgb.predict(X_test)
MAPE(y_test, pred)

11.950171831880308

In [19]:
xgb.fit(X_train_xgb,y)
pred=xgb.predict(X_test_xgb)
submit(pred,"xgb_test_clead")

### CatBoost

In [10]:
X_train, X_test, y_train, y_test=train_test_split(X_train_cat,y,test_size=0.3, random_state=17)

cat=CatBoostRegressor(random_seed=17,loss_function='MAPE')
cat.fit(X_train,np.log(y_train),[0,2,3,4,5,7,8,10,15,16,17],verbose=300)

pred=cat.predict(X_test)
MAPE(y_test, np.exp(pred))

0:	learn: 0.9998336	total: 86.7ms	remaining: 1m 26s
300:	learn: 0.9499614	total: 7.99s	remaining: 18.5s
600:	learn: 0.9000114	total: 14.1s	remaining: 9.38s
900:	learn: 0.8500076	total: 18.1s	remaining: 1.99s


99.998355786818053